# Hopfield Network

A Hopfield network is a somewhat ancient form of neural network who's capabilities have been long surpassed by more modern algorithms. However they have been considered more recently in the context of quantum computing, see https://arxiv.org/abs/1408.7005. The neurons in a Hopfield network take only binary values, usually represented as 1 and -1, which would lend itself to quantum computing much more easily than the more complex activations in modern neural networks. The similarity to the Ising model commonly used in physics also makes this familiar to me.

In a quantum neural network the neurons would be premoted to, bizarrely named even by physics standards, "qurons" which can exist in a superposition of both possible states.

I'm somewhat skeptical about quantum computers being powerful enough to run anything like a neural network in a reasonable amount of time. But still the review above was fun to read and my background in string theory gives me a fairly high tolerance for science fiction.

Unfortunately I haven't got my quantum computer with me right now so I'll just implement this classically for now. As well as practice some object oriented programming while I'm at it.

We'll implement a simple network to reconstruct 5 letter words from a noisy seed. I'll train it with 3 words then test it with some spelling mistakes.

In [6]:
import numpy as np

## Data Processing

In [39]:
def encode_word(word):
    binary_string = "".join(format(ord(x), 'b') for x in word)
    return (2*np.fromiter(binary_string, int) - 1).astype(int)

def decode_word(code):
    assert len(code) % 7 == 0
    num_letters = int(len(code) / 7)
    letters_array = np.split(((code+1)/2).astype(int), num_letters)
    ascii_numbers = []
    for letter in letters_array:
        binary_string = "".join([str(bit) for bit in letter])
        ascii_numbers.append(int(binary_string, 2))
    
    return "".join(chr(ascii_num) for ascii_num in ascii_numbers)

## Model

A Hopfield network is not trained in the modern sense. Since it is so simple (only one layer) we just assign the weights using a rule.

I'll use the Hebbian rule for simplicity but it has been shown that the more complex Storkey rule gives better performance

In [121]:
class hopfield_net:
    
    def __init__(self, size):
        self.size = size
    
    def train(self, train_set, theta_in = None):
        if theta_in:
            assert len(theta_in) == self.size
            self.theta = theta_in
        else:
            self.theta = np.zeros(self.size)
        
        self.w = sum([np.outer(x,x) for x in train_set]) / len(train_set)
    
    def update_state(self, x_in, w, theta):
        return np.dot(w, x_in) > theta
    
    def run_network(self, input_state, num_iters = 10):
        assert len(input_state) == self.size
        x = input_state
        for i in range(num_iters):
            prev_x = x
            x = self.update_state(x, self.w, self.theta)
            if np.sum(x == prev_x) == len(x):
                return x
        
        return x

## "Training"

In [122]:
# Training words
train_words = ["jazzy", "hello", "seven"]
train_words_prep = [encode_word(word) for word in train_words]

In [123]:
word_rememberer = hopfield_net(35)
word_rememberer.train(train_words_prep)

In [124]:
def end_to_end(word_in):
    code_in = encode_word(word_in)
    code_out = word_rememberer.run_network(code_in)
    return decode_word(code_out)

## Testing

In [143]:
end_to_end("forks")

'jazzy'

In [144]:
end_to_end("hello")

'je~lo'

In [146]:
end_to_end("waver")

'seven'

By messing around with this I found that the network had learned "jazzy" and "seven" but not "hello". We also have a false minimum at "je~lo". This is in agreement with the behaviour described in the literature.